# Coded in R

In [1]:
library(WGCNA)
library(dplyr)
setwd("../../data/donors/wgcna/")

In [ ]:
getrows<-function(modules,path){
    rows<-c()
    for(i in modules){
        module.file<-paste0(path,i,".txt")
        module<-read.csv(module.file,header=F,sep="\t")
        rows<-c(rows,module$V1)
    }
    return(rows)
}
add_data_type_to_mtx<-function(df,type){
    idx=c()
    for (i in rownames(df)){
        idx=c(idx,paste0(i,"_",type))
    }
    rownames(df)<-idx
    return(df)
}
#wKDA prep
getmodulesrows<-function(modules,path,type){
    module<-data.frame()
    rows<-c()
    df <- data.frame(matrix(ncol = 2, nrow = 0))
    colnames(df) <- c('MODULE', 'NODE')
    for(i in modules){
        module.file<-paste0(path,i,".txt")
        module<-read.csv(module.file,header=F,sep="\t")
        colnames(module)<-c("NODE")
        module$MODULE=paste0(i,"_",type)
        df<-rbind(df,module)
    }
    
    return(df)
}

generate_nodes_for_mergeomics<-function(modules,path,type,addtype2node=F){
    require(dplyr)
    nodes <- data.frame(matrix(ncol = 2, nrow = 0))
    colnames(nodes)<-c("MODULE","NODE")
    #Make nodes file for mergeomics
    for (i in modules){
        module<-read.csv(paste0(path,i,".txt"),sep="\t",header=F)
        colnames(module)<-"NODE"
        module$MODULE<-i
        module<-module%>%relocate("MODULE")
        nodes<-rbind(nodes,module)
    }
    for (i in 1:nrow(nodes)){
        nodes[i,1]<-paste0(nodes[i,1],"_",type)
        if(addtype2node){
            nodes[i,2]<-paste0(nodes[i,2],"_",type)
        }
    }
    return(nodes)
}

In [ ]:
selected.modules<-read.csv("selected_modules.tsv",sep="\t")

In [ ]:
meta.modules<-selected.modules$metagenome
feces.modules<-selected.modules$feces

In [ ]:
data.meta<-read.csv("../raw_data/metagenome_new.txt",sep="\t",row.names = 1)
data.meta<-log(data.meta+1)
data.feces<-read.csv("../raw_data/metabolomics.txt",sep="\t",row.names=1)

In [ ]:
row.meta<-getrows(meta.modules,"modules/modules_metagenome/")
row.feces<-getrows(feces.modules,"modules/modules_metabolomics/modules/")

In [ ]:
data.meta<-data.meta[row.meta,]
data.feces<-data.feces[row.feces,]
data.meta<-add_data_type_to_mtx(data.meta,"meta")
data.feces<-add_data_type_to_mtx(data.feces,"feces")
data<-rbind(data.meta,data.feces)
write.table(data,file="combined.data.txt",sep="\t",quote=F)

In [ ]:
dam_metab<-read.csv("DAM_metabolite.txt",sep="\t")
dam_metab$NODE<-paste0(dam_metab$NODE,"_feces")
dam_metab$MODULE<-"DEG_feces"
dam_metab<-dam_metab[,c(2,1)]


In [ ]:
nodes.meta<-generate_nodes_for_mergeomics(meta.modules,"modules/modules_metagenome/","meta")
nodes.feces<-generate_nodes_for_mergeomics(feces.modules,"modules/modules_metabolomics/modules/","feces",addtype2node=T)
nodes<-rbind(nodes.meta,nodes.feces,dam_metab)
write.table(nodes,"Node4wKDA.txt",sep="\t",row.name=F,quote=F)